# Comparison of calculation results for Europian option

## $OU$ model case

### Theoretical value

\begin{align}
    & X_t \sim N(x_0 e^{-\alpha t}, \frac{\sigma^2}{2\alpha} (1 - e^{-2 \alpha t})) \\
    & \hat{\mu} := x_0 e^{-\alpha t}, \hat{\sigma}^2 := \frac{\sigma^2}{2\alpha} (1 - e^{-2 \alpha t}) \\
    & f(x) = (e^x - K)_+
\end{align}

なので、

\begin{align}
    \mathbb{E}[f(X_t)] = e^{\hat{\mu} + \frac{\hat{\sigma}^2}{2}} \mathbb{P}
    \left( \frac{\log{K}-(\hat{\mu} + \hat{\sigma}^2)}{\hat{\sigma}} \le N(0,1) \right)
    - K \mathbb{P} \left( \frac{\log{K}- \hat{\mu}}{\hat{\sigma}} \le N(0,1) \right)
\end{align}

In [47]:
import scipy
from scipy.stats import norm
import numpy as np

In [48]:
import parameter
x_0 = parameter.x_0
alpha = parameter.alpha
sigma = parameter.sigma
K = parameter.K
n = 100
T = parameter.T

import math
e = math.e

In [49]:
mu_hat = x_0 * e**(- alpha * T)
sigma_hat = math.sqrt(sigma**2 / (2*alpha) * (1 - e**(-2*alpha*T)))

In [50]:
sigma_hat_p2 = sigma**2  * (1 - np.exp(- 2 * alpha * T)) / ( 2 * alpha)
print(sigma_hat_p2)

2.00420917795792


In [51]:
print(norm.sf((np.log(K) - mu_hat - sigma_hat**2)/sigma_hat, loc=0, scale=1))
print(norm.sf(np.log(K), loc= mu_hat + sigma_hat**2, scale=np.sqrt(sigma_hat**2)))

0.9946531180419257
0.9946531180419257


In [52]:
theoretical_price_OU_model_case = e**(mu_hat + sigma_hat**2 /2) * norm.sf((K - mu_hat - sigma_hat**2)/sigma_hat, loc=0, scale=1) - K * norm.sf((K - mu_hat)/sigma_hat, loc=0, scale=1)

In [53]:
print(theoretical_price_OU_model_case)

2.3594214381470935


In [54]:
import numpy as np
from statistics import mean

In [55]:
import OU_model_generator

## discrete $OU$ model case

In [56]:
import discrete_OU_model_generator

In [57]:
delta = discrete_OU_model_generator.Delta(n, T, sigma)

def weight_deltas(k):
    return [i * delta for i in range (-k, k+1, 2)]

p = discrete_OU_model_generator.up_probability

In [58]:
sum_probability_dic = {}
sum_probability = None
def sum_probability_for_weight(k=3, position = 4):
    if k in sum_probability_dic.keys():
        if position in sum_probability_dic[k].keys():
            #print('skip')
            pass
        else:
            if k == 1:
                if position == 0:
                    sum_probability = 1 - p(alpha, x_0)
                else:
                    sum_probability = p(alpha, x_0)     
            elif position == k:
                sum_probability = p(alpha, x_0 + weight_deltas(k-1)[position - 1]) * sum_probability_for_weight(k-1, position - 1)
            elif position == 0:
                sum_probability = (1 - p(alpha, x_0 + weight_deltas(k-1)[0])) * sum_probability_for_weight(k-1, 0)
            else:
                if k-1 in sum_probability_dic.keys():
                    if position in sum_probability_dic[k-1].keys():
                        #print('skip')
                        pass
                    else:
                        sum_probability1 = sum_probability_for_weight(k-1, position)
                        sum_probability_dic[k-1][position] = sum_probability1
                    if position - 1 in sum_probability_dic[k-1].keys():
                        #print('skip')
                        pass
                    else:
                        sum_probability2 = sum_probability_for_weight(k-1, position - 1)
                        sum_probability_dic[k-1][position - 1] = sum_probability2
                else:
                    sum_probability1 = sum_probability_for_weight(k-1, position)
                    sum_probability_dic[k-1] = {position : sum_probability1}
                    sum_probability2 = sum_probability_for_weight(k-1, position - 1)
                    sum_probability_dic[k-1][position - 1] = sum_probability2
                sum_probability = p(alpha, x_0 + weight_deltas(k-1)[position - 1]) * sum_probability_dic[k-1][position - 1] + (1 - p(alpha, x_0 + weight_deltas(k-1)[position])) * sum_probability_dic[k-1][position]
            sum_probability_dic[k][position] = sum_probability
    
    else:
        if k == 1:
            if position == 0:
                sum_probability = 1 - p(alpha, x_0)
            else:
                sum_probability = p(alpha, x_0)  
        elif position == k:
            sum_probability = p(alpha, x_0 + weight_deltas(k-1)[position - 1]) * sum_probability_for_weight(k-1, position - 1)
        elif position == 0:
            sum_probability = (1 - p(alpha, x_0 + weight_deltas(k-1)[0])) * sum_probability_for_weight(k-1, 0)
        else:
            sum_probability1 = sum_probability_for_weight(k-1, position)
            sum_probability_dic[k-1] = {position : sum_probability1}
            sum_probability2 = sum_probability_for_weight(k-1, position - 1)
            sum_probability_dic[k-1][position - 1] = sum_probability2
            sum_probability = p(alpha, x_0 + weight_deltas(k-1)[position - 1]) * sum_probability_dic[k-1][position - 1] + (1 - p(alpha, x_0 + weight_deltas(k-1)[position])) * sum_probability_dic[k-1][position]

        sum_probability_dic[k] = {position : sum_probability}
    
    return sum_probability_dic[k][position]

In [ ]:
sum_probability_dic = {}
k = n
for i in range(k+1):
    sum_probability_for_weight(k,i)
#print(sum_probability_dic)
print(weight_deltas(k))

[-23.0, -22.54, -22.08, -21.619999999999997, -21.159999999999997, -20.7, -20.24, -19.779999999999998, -19.32, -18.86, -18.4, -17.939999999999998, -17.479999999999997, -17.02, -16.56, -16.099999999999998, -15.639999999999999, -15.18, -14.719999999999999, -14.259999999999998, -13.799999999999999, -13.34, -12.879999999999999, -12.419999999999998, -11.959999999999999, -11.5, -11.04, -10.579999999999998, -10.12, -9.66, -9.2, -8.739999999999998, -8.28, -7.819999999999999, -7.359999999999999, -6.8999999999999995, -6.4399999999999995, -5.9799999999999995, -5.52, -5.06, -4.6, -4.14, -3.6799999999999997, -3.2199999999999998, -2.76, -2.3, -1.8399999999999999, -1.38, -0.9199999999999999, -0.45999999999999996, 0.0, 0.45999999999999996, 0.9199999999999999, 1.38, 1.8399999999999999, 2.3, 2.76, 3.2199999999999998, 3.6799999999999997, 4.14, 4.6, 5.06, 5.52, 5.9799999999999995, 6.4399999999999995, 6.8999999999999995, 7.359999999999999, 7.819999999999999, 8.28, 8.739999999999998, 9.2, 9.66, 10.12, 10.579

In [ ]:
for k in range(2):
    print(k)
    print(sum([sum_probability_for_weight(alpha, x_0, k, x_0 + increment) for increment in weight_deltas(k)]))

0


TypeError: sum_probability_for_weight() takes from 0 to 2 positional arguments but 4 were given

In [ ]:
def mean_discrete_OU_process(n=100, T=1, time=0.01, alpha=1.5, sigma=1.2, z_0=0):
    k = (int) (n * time)
    weights = weight_deltas(k)
    probabilities = [sum_probability_for_weight(alpha, z_0, k, z_0 + increment) for increment in weight_deltas(k)]
    return sum([weight * p for weight, p in zip(weights, probabilities)]),[weight * p for weight, p in zip(weights, probabilities)]

In [ ]:
mean_discrete_OU_process(n=10, T=1, time=0.6, alpha=1.5, sigma=1.5, z_0=1.2)[0]

0.0

In [ ]:
n = 1  # 切り捨てしたい桁
x = 1.555
y = np.floor(x * 10 ** n) / (10 ** n)
print(x, "=>", y)

1.555 => 1.5
